In [8]:
import torch
import torch.nn as nn

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.leaky_relu = nn.LeakyReLU(0.1)

    def forward(self, x):
        return self.leaky_relu(self.bn(self.conv(x)))

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
        ConvBlock(channels, channels // 2, kernel_size=1, stride=1, padding=0),
        ConvBlock(channels // 2, channels, kernel_size = 3, stride = 1, padding = 1)
        )
    def forward(self, x):
        return x + self.block(x)

class Darknet53(nn.Module):
    def __init__(self):
        super(Darknet53, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(
        ConvBlock(3, 32, kernel_size=3, stride=1, padding=1))
        self.layers.append(
        ConvBlock(32, 64, kernel_size=3, stride=2, padding=1))
        self.layers.append(ResidualBlock(64))
        self.layers.append(
        ConvBlock(64, 128, kernel_size=3, stride=2, padding=1))
        for _ in range(2):
            self.layers.append(ResidualBlock(128))
        self.layers.append(
        ConvBlock(128, 256, kernel_size=3, stride=2, padding=1))
        for _ in range(8):
            self.layers.append(ResidualBlock(256))
        self.layers.append(
        ConvBlock(256, 512, kernel_size=3, stride=2, padding=1))
        for _ in range(8):
            self.layers.append(ResidualBlock(512))
        self.layers.append(
        ConvBlock(512, 1024, kernel_size=3, stride=2, padding=1))
        for _ in range(4):
            self.layers.append(ResidualBlock(1024))
        self.layers.append(
        ConvBlock(1024, 1024, kernel_size=1, stride=1, padding=0))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

if __name__ == "__main__":
    model = Darknet53()
    # Sample input tensor with batch size of 1 and image size 416x416
    x = torch.randn(1, 3, 416, 416)
    output = model(x)
    print(output.shape) # Expected output shape: (1, 1024, 13, 13)

torch.Size([1, 1024, 13, 13])


In [13]:
import cv2
import torch
import numpy as np
from pathlib import Path

# Install ultralytics (YOLOv3 pretrained available)
# !pip install ultralytics

from ultralytics import YOLO

# Load YOLOv3 (Darknet-53 backbone is default in YOLOv3)
# Pretrained on COCO dataset
model = YOLO("yolov3.pt")  # Or replace with "yolov3u.pt" for better performance

# Initialize webcam (0 = default laptop camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Error: Could not open camera.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to grab frame.")
        break

    # Run YOLOv3 inference
    results = model(frame)

    # Annotate results on frame
    annotated_frame = results[0].plot()

    # Show the annotated frame
    window_name = "YOLOv3 Object Detection - Darknet-53"
    cv2.imshow(window_name, annotated_frame)

    # Exit if 'q' is pressed OR window is closed
    if (cv2.waitKey(1) & 0xFF == ord("q")) or cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
cv2.destroyAllWindows()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Acer\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Acer\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\Acer\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "C:\Users\Acer\anaconda3\Lib\site-packages

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [11]:
!pip install opencv-python

  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
